<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/notebooks/RAG_ParentDocumentRetriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install langchain chromadb --quiet
%pip -q install sentence_transformers --quiet
%pip -q install -U FlagEmbedding --quiet
%pip install huggingface_hub --quiet
%pip install -q -U transformers peft accelerate optimum --quiet
%pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --quiet
%pip install llama-index --quiet
%pip install GPUtil --quiet
%pip install --quiet unstructured
%pip install --upgrade --quiet  langchain langsmith langchainhub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain import PromptTemplate
import torch
import GPUtil
import pandas as pd

In [3]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "RAG_2_PArentDocumentRetriever"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__40df14b1c671470a963cb1fad30cb872"

In [4]:
from langsmith import Client
client = Client()

In [5]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

BGE models on the HuggingFace are the best open-source embedding models. BGE model is created by the Beijing Academy of Artificial Intelligence (BAAI). BAAI is a private non-profit organization engaged in AI research and development.

In [6]:
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'}, #cpu
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/93.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

File Directory
This covers how to load all documents in a directory.

Under the hood, by default this uses the UnstructuredLoader.

In [7]:
%%capture
!wget 'https://raw.githubusercontent.com/winterForestStump/thesis/main/notebooks/example_texts/txt_examp_1.txt' -O 'txt_examp_1.txt'
!wget 'https://raw.githubusercontent.com/winterForestStump/thesis/main/notebooks/example_texts/txt_examp_2.txt' -O 'txt_examp_2.txt'
!wget 'https://raw.githubusercontent.com/winterForestStump/thesis/main/notebooks/example_texts/txt_examp_3.txt' -O 'txt_examp_3.txt'

In [8]:
loader = DirectoryLoader('/content', glob="**/*.txt", show_progress=True)

docs = loader.load()

  0%|          | 0/3 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 3/3 [00:29<00:00,  9.99s/it]


In [9]:
len(docs)

3

When splitting documents for retrieval, there are often conflicting desires:

You may want to have small documents, so that their embeddings can most accurately reflect their meaning. If too long, then the embeddings can lose meaning.
You want to have long enough documents that the context of each chunk is retained.
The ParentDocumentRetriever strikes that balance by splitting and storing small chunks of data. During retrieval, it first fetches the small chunks but then looks up the parent ids for those chunks and returns those larger documents.

Note that “parent document” refers to the document that a small chunk originated from. This can either be the whole raw document OR a larger chunk.

Sometimes, the full documents can be too big to want to retrieve them as is. In that case, what we really want to do is to first split the raw documents into larger chunks, and then split it into smaller chunks. We then index the smaller chunks, but on retrieval we retrieve the larger chunks (but still not the full documents).

In [10]:
# This text splitter is used to create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

# This text splitter is used to create the child documents - The small chunks
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=256)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="split_parents", embedding_function=bge_embeddings)

# The storage layer for the parent documents
store = InMemoryStore()

In [11]:
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [12]:
big_chunks_retriever.add_documents(docs)

We can see that there are much more than two documents now - these are the larger chunks.

In [13]:
len(list(store.yield_keys()))

612

Let’s make sure the underlying vector store still retrieves the small chunks.

In [14]:
sub_docs = vectorstore.similarity_search("What is the total revenue (sales) of the Air Products and Chemicals, Inc.?")

In [15]:
len(sub_docs)

4

In [16]:
print(sub_docs[0].page_content)

Air Products and Chemicals, Inc. and Subsidiaries

CONSOLIDATED INCOME STATEMENTS

For the Fiscal Years Ended 30 September

(Millions of U.S. Dollars, except for share and per share data) 2023 2022 2021

Sales $12,600.0 $12,698.6 $10,323.0


In [17]:
retrieved_docs = big_chunks_retriever.get_relevant_documents("What is the total revenue (sales) of the Air Products and Chemicals, Inc.?")

In [18]:
len(retrieved_docs)

4

In [19]:
len(retrieved_docs[0].page_content)

1991

In [20]:
print(retrieved_docs[0].page_content)

Philadelphia, Pennsylvania

November 16, 2023

We have served as the Company's auditor since 2018.

Air Products and Chemicals, Inc. and Subsidiaries

CONSOLIDATED INCOME STATEMENTS

For the Fiscal Years Ended 30 September

(Millions of U.S. Dollars, except for share and per share data) 2023 2022 2021

Sales $12,600.0 $12,698.6 $10,323.0

Cost of sales 8,833.0 9,338.5 7,186.1

Facility closure - - 23.2

Selling and administrative expense

957.0 900.6 828.4

Research and development expense

105.6 102.9 93.5

Business and asset actions 244.6 73.7 -

Gain on exchange with joint venture partner - - 36.8

Other income (expense), net 34.8 55.9 52.8

Operating Income 2,494.6 2,338.8 2,281.4

Equity affiliates' income 604.3 481.5 294.1

Interest expense 177.5 128.0 141.8

Other non-operating income (expense), net (39.0) 62.4 73.7

Income From Continuing Operations Before Taxes 2,882.4 2,754.7 2,507.4

Income tax provision 551.2 500.8 462.8

Income From Continuing Operations 2,331.2 2,253.9 2,

In [21]:
print(retrieved_docs[1].page_content)

Total Air Products Shareholders' Equity 14,312.9 13,144.0

Noncontrolling Interests(A)

1,347.4 558.4

Total Equity 15,660.3 13,702.4

Total Liabilities and Equity $32,002.5 $27,192.6

(A)Includes balances associated with a consolidated variable interest entity ("VIE"), including amounts reflected in "Total Assets" that can only be used to settle obligations of the VIE of $2,256.8 and $519.7 as of 30 September 2023 and 30 September 2022, respectively, as well as liabilities of the VIE reflected within "Total Liabilities" for which creditors do not have recourse to the general credit of Air Products of $1,461.1 and $506.8 as of 30 September 2023 and 30 September 2022, respectively. Refer to Note 3, Variable Interest Entities, for additional information.

The accompanying notes are an integral part of these statements.

Air Products and Chemicals, Inc. and Subsidiaries

CONSOLIDATED STATEMENTS OF CASH FLOWS

For the Fiscal Years Ended 30 September

(Millions of U.S. Dollars) 2023 2022 20

In [22]:
MODEL_NAME = "TheBloke/Llama-2-7b-Chat-GPTQ"
TEMPERATURE = 0.0001
MAX_NEW_TOKENS = 2048
TOP_P = 0.90
REPETITION_PENALTY = 1.10


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto")

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = MAX_NEW_TOKENS
generation_config.temperature = TEMPERATURE
generation_config.top_p = TOP_P
generation_config.do_sample = True
generation_config.repetition_penalty = REPETITION_PENALTY

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": TEMPERATURE})

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [23]:
# K = 3
# SCORE_THRESHOLD = 0.2


template = """
<s>[INST] <<SYS>>
Use the following information from company annual reports and answer the questions at the end.
If the answer is not contained in the provided information or if there is NO context at all, say "The answer is not in the context".
Do not use phrases that "based on the information provided", etc in the answer.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= big_chunks_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

print('GPU Usage:')
GPUtil.showUtilization()

GPU Usage:
| ID | GPU | MEM |
------------------
|  0 |  0% | 28% |


In [24]:
questions = pd.read_fwf("https://raw.githubusercontent.com/winterForestStump/thesis/main/questions_ver2.txt", names=['question'])

In [25]:
# Here you need to enter the company name
company = 'Air Products and Chemicals, Inc.'

questions['question'] = questions['question'].str.replace('company', company)
# questions['Answer'] = questions['Questions'].apply(lambda x: qa_chain(x)['result'].strip())

def extract_info(query):
    result = qa_chain(query)
    print('GPU Usage:')
    GPUtil.showUtilization()
    return result['result'].strip(), list(result['source_documents'])

questions[['answer', 'contexts']] = questions['question'].apply(lambda x: pd.Series(extract_info(x)))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 98% | 56% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 78% | 56% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 92% | 56% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 92% | 56% |


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 58% | 68% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 81% | 68% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 98% | 68% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 64% | 68% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 98% | 77% |
GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 88% | 77% |


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 98% | 97% |


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 97% | 97% |


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


GPU Usage:
| ID | GPU | MEM |
------------------
|  0 | 98% | 97% |


In [26]:
questions.to_json('QA_ver8.json', orient="records")

In [27]:
len(questions['contexts'])

35

In [28]:
questions

,question,answer,contexts
0,What is the total revenue generated by the Air...,Based on the information provided in the annua...,"[page_content=""Philadelphia, Pennsylvania\n\nN..."
1,"What is the Air Products and Chemicals, Inc.'s...",Based on the information provided in the annua...,"[page_content=""Philadelphia, Pennsylvania\n\nN..."
2,"What is the gross profit margin, and how has i...",Based on the information provided in the annua...,[page_content='Cost of Sales and Gross Margin\...
3,What are the major operating expenses and how ...,"According to the provided information, the maj...",[page_content='19.9 19.1\n\nWeighted-average d...
4,"What is the Air Products and Chemicals, Inc.'s...",According to the information provided in the a...,[page_content='Total Air Products Shareholders...
5,What is the net income for the current fiscal ...,Based on the information provided in the annua...,[page_content='Nonoperating Expense (Income)\n...
6,What is the earnings per share and how does it...,Based on the information provided in the annua...,[page_content='The Company recognizes the effe...
7,What is the cash flow generated from operation...,"According to the provided information, the com...",[page_content='Principal amount of term loans ...
8,"How much has the Air Products and Chemicals, I...",Based on the information provided in the annua...,[page_content='Total Air Products Shareholders...
9,"What is the total outstanding debt, how is the...",Based on the information provided in the annua...,[page_content='Assets at Fair Value\n\nDerivat...
